In [ ]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

     |████████████████████████████████| 138 kB 6.8 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=f1d0e1cd32fefc30f34bbed524c2930a257be2b66d3fb5fcce030f2a0672541d
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [ ]:
! signate list
! signate files --competition-id=492
! signate download --competition-id=492

  competitionId  title                                                                  closing     prize            submitters
---------------  ---------------------------------------------------------------------  ----------  -------------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                   -                                  5137
             24  【練習問題】お弁当の需要予測                                           -                                  6321
             27  【練習問題】Jリーグの観客動員数予測                                    -                                  1431
            100  【練習問題】手書き文字認識                                             -           Knowledge               161
            102  【練習問題】タイタニックの生存予測                                     -           Knowledge              1055
            103  【練習問題】音楽ラベリング                                             -           Knowledge                60
            104  【練習問題】スパムメール分類                                     

In [ ]:
# ディレクトリ内のデータの確認
%ls

adc.json                                       requirements.txt
category_names.csv                             sales_history.csv
item_categories.csv                            sample_data/
PBL01_h_exercise4_question.pdf                 sample_submission.csv
PBL01_i_exercise4_supplementary-documents.pdf  test.csv
PBL01_sample_code.ipynb


In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# データの読み込み
category_names = pd.read_csv('category_names.csv')
item_categories = pd.read_csv('item_categories.csv')
sales_history = pd.read_csv('sales_history.csv')

'''
結合データの作成
'''

# データの結合
productID_category = pd.merge(category_names, item_categories, on='商品カテゴリID', how='left')

# 学習用データの作成
merged_df = pd.merge(productID_category, sales_history, on='商品ID', how='left')

# カラム名の変更
merged_df = merged_df.rename(columns={'商品カテゴリID':'category_id',
                                      '商品カテゴリ名':'category_name',
                                      '商品ID': 'id',
                                      '日付':'datetime',
                                      '店舗ID':'store_id',
                                      '商品価格':'price',
                                      '売上個数':'sales'
                                      })

# データの確認
print(merged_df.dtypes)

# datetime型への変換
merged_df['datetime'] = pd.to_datetime(merged_df['datetime'], format='%Y-%m-%d')

# 西暦、月、日、曜日カラムの作成
merged_df['year'] = merged_df['datetime'].dt.strftime("%Y")
merged_df['month'] = merged_df['datetime'].dt.strftime("%m")

'''
月でビニング
'''

# 結合データの「年」と「月」をグルーピングして、「年」と「月」の全組み合わせを作成する
time_group = merged_df.groupby(['year', 'month']).count().reset_index()[['year', 'month']]

# カラム名「month_bining」として、通し番号をつける
time_group['month_bining'] = list(range(len(time_group)))

# testデータ用に、評価対象期間である2019年12月(month_bining: 23)のレコードを追加する
time_group = time_group.append({'year':'2019', 'month':'12', 'month_bining':23}, ignore_index=True)

# time_groupをCSVファイルに出力
time_group.to_csv('/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/processed_data/time_group.csv', header=True, index=False)

'''
結合データに「month_bining」カラムを付与したデータフレームを作成（merged_df2）
'''

merged_df2 = pd.merge(merged_df, time_group, on=['year', 'month'], how='left')
print(merged_df2.head())
print(merged_df2.tail())

'''
月ごとの売上の集計(「商品ID」×「店舗ID」)
'''

# 「month_bining」「id」「store_id」でグルーピングし、売上個数の合計を取る
monthly_sales = merged_df2.groupby(['month_bining', 'id', 'store_id']).agg({'sales': np.sum}).reset_index()
print(monthly_sales.head())

'''
「'月'、'商品ID'、 '店舗ID'」の全組み合わせの作成
'''

# ライブラリのインポート
from itertools import product

# itertools.product関数を使ってすべての組み合わせをデータフレーム化
all_combination = []
for i in range(22):
    train_block = merged_df2[merged_df2['month_bining']==i]
    all_combination.append(np.array(list(product([i], train_block['id'].unique(), train_block['store_id'].unique()))))
    
all_combination = pd.DataFrame(np.vstack(all_combination), columns=['month_bining', 'id', 'store_id'])
all_combination.sort_values(['month_bining', 'id', 'store_id'], inplace=True)

print(all_combination.info())

'''
all_combinationに月ごとの売上データ（monthly_sales）を結合
'''

pre_train = pd.merge(all_combination, monthly_sales, on=['month_bining', 'id', 'store_id'], how='left')

# 売上個数がNaNとなっている箇所は、売上個数0個として値を埋める
pre_train['sales'] = pre_train['sales'].fillna(0)

'''
pre_trainデータをCSVファイルとして出力
'''

pre_train.to_csv('/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/processed_data/pre_train.csv', header=True, index=False)

category_id        int64
category_name     object
id                 int64
datetime          object
store_id         float64
price            float64
sales            float64
dtype: object
   category_id category_name       id  ...  year  month  month_bining
0          100      映画 - DVD  1000001  ...  2018     01           0.0
1          100      映画 - DVD  1000001  ...  2018     01           0.0
2          100      映画 - DVD  1000001  ...  2018     01           0.0
3          100      映画 - DVD  1000001  ...  2018     01           0.0
4          100      映画 - DVD  1000001  ...  2018     01           0.0

[5 rows x 10 columns]
         category_id      category_name       id  ...  year  month  month_bining
1119951          350  本 - オーディオブック_タイプB  3500072  ...  2019     10          21.0
1119952          350  本 - オーディオブック_タイプB  3500073  ...  2019     10          21.0
1119953          350  本 - オーディオブック_タイプB  3500074  ...   NaN    NaN           NaN
1119954          350  本 - オーディオブック_タイプB  350